### 3.7 upload to aws s3

In [19]:
import logging
import boto3
from botocore.exceptions import ClientError

s3c = boto3.client('s3')
s3r = boto3.resource('s3')

+ get the list of buckets from aws s3

In [20]:
buckets = []

# Print out bucket names
for bucket in s3r.buckets.all():
    buckets.append(bucket)

ClientError: An error occurred (InvalidAccessKeyId) when calling the ListBuckets operation: The AWS Access Key Id you provided does not exist in our records.

+ define the method to upload a file to an AWS bucket.

In [ ]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

+ upload the file to the first bucket from aws s3
+ it will directly overide the file with the same name in the directory.

In [ ]:
print(upload_file('meetingSticker.db',buckets[0].name))

+ download the file from AWS S3 first bucket.

In [ ]:
print(s3c.download_file(buckets[0].name,'meetingSticker.db',local_file))

### initiate amazon dynamodb connection

In [4]:
import boto3

dynDBc = boto3.client('dynamodb')
dynDBr = boto3.resource('dynamodb')

In [ ]:


# Create the DynamoDB table.
table = dynDBr.create_table(
    TableName='users',
    KeySchema=[
        {
            'AttributeName': 'username',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'last_name',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'userID',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'username',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'password',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'first_name',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'last_name',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'email',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'phone',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='users')


### initiate amazon s3 connection

In [ ]:
# Let's use Amazon S3

import boto3
import json
import pandas as pd

s3c = boto3.client('s3')
s3r = boto3.resource('s3')

### load all buckets from s3 and second bucket's objects

In [ ]:
buckets = []

# Print out bucket names
for bucket in s3r.buckets.all():
    buckets.append(bucket)

+ an iterator to go through all the objects from bucket[1] s3 client.

In [ ]:
for i in buckets[1].objects.iterator():
    pass
    #print(i)

+ get all the document/object from second bucket form the bucket list.

In [ ]:
documents = s3c.list_objects(Bucket = buckets[1].name)['Contents']
textract_c = boto3.client('textract')

documents_ls = [d['Key'] for d in documents]
#documents_ls

### get responses and extract their contents from amazon textract

In [ ]:
response_ls = []
for d in documents_ls:
    response_ls.append(textract_c.start_document_analysis(DocumentLocation={'S3Object':{'Bucket':buckets[1].name,'Name':d}},FeatureTypes = ['TABLES']))
    
jobId_ls = [r['JobId'] for r in response_ls]

In [ ]:
result_ls = []

for j in jobId_ls:
    result_ls.append(textract_c.get_document_analysis(JobId=j))

In [ ]:
jobStatus = [r['JobStatus'] for r in result_ls]
jobStatus_df = pd.DataSeries(jobStatus)

### Saving each raw text in result_ls to .json

In [ ]:
file_head = 'city of san jose minutes_'
file_surfix = '.json'
i = 0

for r in result_ls:
    raw_txt,raw_ls = creatRaw(result_ls[i])
    with open(file_head+str(i+1)+file_surfix,'w') as f:
        json.dump(result_ls[i],f)
        f.close()
    i+=1
    

### create raw text and raw content list from textract string input.

In [ ]:
def creatRaw(result):

    raw_ls=[]
    raw_txt=''
    for t in result['Blocks']:
        try:
            if t['Text'][:4]=='Page':
                pass
            else:
                raw_ls.append( t['Text'])
                raw_txt += t['Text']+' '
        except:
            pass

    return raw_txt,raw_ls

### load and create raw text file from .json file.

In [ ]:
import json

with open('./raw/city of san jose minutes_19.json') as jsonFile:
    data = json.load(jsonFile)
    
rawtxt, rawls = creatRaw(data)